In [32]:
import pandas as pd
import numpy as np

In [33]:
df = pd.read_csv(r'fraud_detection.csv')
df.head()

,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,city,...,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud,merch_zipcode,hour,age
0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,Moravian Falls,...,"Psychologist, counselling",1988-03-09,0b242abb623afc578575680df30655b9,1325376018,36.011293,-82.048315,0,28705.0,0,32
1,2019-01-01 00:00:51,38859492057661,fraud_Lind-Buckridge,entertainment,220.11,Edward,Sanchez,M,594 White Dale Suite 530,Malad City,...,Nature conservation officer,1962-01-19,a1a22d70485983eac12b5b88dad1cf95,1325376051,43.150704,-112.154481,0,83236.0,0,58
2,2019-01-01 00:03:06,375534208663984,fraud_Keeling-Crist,misc_pos,41.96,Tyler,Garcia,M,408 Bradley Rest,Doe Hill,...,Dance movement psychotherapist,1986-03-28,a41d7549acf90789359a9aa5346dcb46,1325376186,38.674999,-78.632459,0,22844.0,0,34
3,2019-01-01 00:04:08,4767265376804500,"fraud_Stroman, Hudson and Erdman",gas_transport,94.63,Jennifer,Conner,F,4655 David Island,Dublin,...,Transport planner,1961-06-19,189a841a0a8ba03058526bcfe566aab5,1325376248,40.653382,-76.152667,0,17972.0,0,59
4,2019-01-01 00:05:08,6011360759745864,fraud_Corwin-Collins,gas_transport,71.65,Steven,Williams,M,231 Flores Pass Suite 720,Edinburg,...,"Designer, multimedia",1947-08-21,6d294ed2cc447d2c71c7171a3d54967c,1325376308,38.948089,-78.540296,0,22644.0,0,72


In [34]:
df['is_fraud'].value_counts()

is_fraud
0    1094339
1       6363
Name: count, dtype: int64

Random Undersampling (Rastgele Azaltma) yöntemiyle is_fraud sınıflarını dengeler. Büyük sınıftan (is_fraud = 0) rastgele örnekler alarak küçük sınıfla (is_fraud = 1) eşit hale getirir:

In [35]:
# Sınıf 0 ve sınıf 1 verilerini ayır
class_0 = df[df['is_fraud'] == 0].sample(n=6363, random_state=42)  # Küçük sınıfın sayısına eşit örnek al
class_1 = df[df['is_fraud'] == 1]  # Küçük sınıfı olduğu gibi al

# İki sınıfı birleştir
df = pd.concat([class_0, class_1])

# Veri setini karıştır
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

df['is_fraud'].value_counts()

is_fraud
0    6363
1    6363
Name: count, dtype: int64

In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12726 entries, 0 to 12725
Data columns (total 25 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   trans_date_trans_time  12726 non-null  object 
 1   cc_num                 12726 non-null  int64  
 2   merchant               12726 non-null  object 
 3   category               12726 non-null  object 
 4   amt                    12726 non-null  float64
 5   first                  12726 non-null  object 
 6   last                   12726 non-null  object 
 7   gender                 12726 non-null  object 
 8   street                 12726 non-null  object 
 9   city                   12726 non-null  object 
 10  state                  12726 non-null  object 
 11  zip                    12726 non-null  int64  
 12  lat                    12726 non-null  float64
 13  long                   12726 non-null  float64
 14  city_pop               12726 non-null  int64  
 15  jo

In [37]:
# drop unnecessary columns
unnecessary_columns = ['trans_date_trans_time', 'cc_num', 'first', 'last', 'trans_num', 'dob']
df.drop(unnecessary_columns, axis=1, inplace=True)

🧱 Kullanılmaması Gereken veya Dikkat Edilmesi Gereken Sütunlar

* cc_num	Benzersiz ama anonim kart numarası → yüksek kardinalite, işe yaramaz.
* trans_num	Benzersiz ID → model öğrenemez.
* first, last, street	Öznitelik olarak anlamlı değil, kişisel veri → çıkartılmalı.
* dob, trans_date_trans_time	Doğrudan kullanılmaz, ama yaş ve saat gibi türevler çıkarılabilir.
* merchant, merch_zipcode, zip, state, city	Kodlanırsa anlam kazanabilir (örn. frequency encoding).

In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12726 entries, 0 to 12725
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   merchant       12726 non-null  object 
 1   category       12726 non-null  object 
 2   amt            12726 non-null  float64
 3   gender         12726 non-null  object 
 4   street         12726 non-null  object 
 5   city           12726 non-null  object 
 6   state          12726 non-null  object 
 7   zip            12726 non-null  int64  
 8   lat            12726 non-null  float64
 9   long           12726 non-null  float64
 10  city_pop       12726 non-null  int64  
 11  job            12726 non-null  object 
 12  unix_time      12726 non-null  int64  
 13  merch_lat      12726 non-null  float64
 14  merch_long     12726 non-null  float64
 15  is_fraud       12726 non-null  int64  
 16  merch_zipcode  12726 non-null  float64
 17  hour           12726 non-null  int64  
 18  age   

# 🛠️ Feature Engineering


In [39]:
# is_high_amount (Yüksek Tutar):
# İşlem tutarının belirli bir eşik değerin üzerinde olup olmadığını belirten bir özellik oluşturabilirsiniz.

In [40]:
df['is_night_transaction'] = df['hour'].apply(lambda x: 1 if x >= 22 or x <= 6 else 0)


is_night_transaction (Gece İşlemi):

İşlemin gece saatlerinde (örneğin, 22:00-06:00) yapılıp yapılmadığını belirten bir özellik oluşturabilirsiniz.

In [41]:
df['transaction_speed'] = df['unix_time'].diff()

transaction_speed (İşlem Hızı):

İşlemin yapıldığı zaman ile önceki işlem arasındaki süreyi hesaplayabilirsiniz. Çok hızlı yapılan işlemler dolandırıcılık göstergesi olabilir

In [42]:
from geopy.distance import geodesic
df['distance_to_merchant'] = df.apply(lambda row: geodesic((row['lat'], row['long']), (row['merch_lat'], row['merch_long'])).km, axis=1)

distance_to_merchant (Satıcıya Uzaklık):

Kullanıcının adresi (lat, long) ile satıcının adresi (merch_lat, merch_long) arasındaki mesafeyi hesaplayabilirsiniz.

In [43]:
df.drop(['lat', 'long', 'merch_lat', 'merch_long'], axis=1, inplace=True)

In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12726 entries, 0 to 12725
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   merchant              12726 non-null  object 
 1   category              12726 non-null  object 
 2   amt                   12726 non-null  float64
 3   gender                12726 non-null  object 
 4   street                12726 non-null  object 
 5   city                  12726 non-null  object 
 6   state                 12726 non-null  object 
 7   zip                   12726 non-null  int64  
 8   city_pop              12726 non-null  int64  
 9   job                   12726 non-null  object 
 10  unix_time             12726 non-null  int64  
 11  is_fraud              12726 non-null  int64  
 12  merch_zipcode         12726 non-null  float64
 13  hour                  12726 non-null  int64  
 14  age                   12726 non-null  int64  
 15  is_night_transactio

In [45]:
df.drop(['zip', 'merch_zipcode', 'unix_time', 'street', 'merchant'], axis=1, inplace=True)

In [46]:
df.to_csv(r'C:\Users\sakir\OneDrive\Masaüstü\career\akbank\globalaihub2\fraud_detection2.csv', index=False)